In [1]:
import math
import time
import copy
from os import system
import numpy as np

In [ ]:
# Class that represents the game Attax
class AttaxGame:
    def __init__(self):
        # Initialize constants and game-related attributes
        self.nMovs = 0 # Number of moves
        self.N = 0 # Board dimension
        self.action_size = 0 # Number of possible actions

    # Class that represents a movement
    class mov: 
        def __init__(self, xi, yi, xf, yf, player, type):
            self.xi = xi # Initial x coordinate
            self.yi = yi # Initial y coordinate
            self.xf = xf # Final x coordinate
            self.yf = yf # Final y coordinate
            self.player = player # Player that made the movement
            self.type = type # Type of movement (0 - expansion, 1 - jump)

    # Movement function
    def movement(self, xi=0, yi=0, xf=0, yf=0, player=0, tipo=0):
        return self.mov(xi, yi, xf, yf, player, tipo) # Returns a movement
    
    # Changes the player
    @staticmethod
    def change_player(player):
        if player == 1: 
            return -1
        else:
            return 1

    # Gets the initial state of the board
    def get_initial_state(self):
        state = np.zeros((self.N, self.N), dtype=int) # Initializes the board with zeros (empty)
        state[0][0] = 1 # Player 1
        state[0][self.N-1] = -1 # Player 2
        state[self.N-1][0] = -1 # Player 2
        state[self.N-1][self.N-1] = 1 # Player 1
        
        return state
        
    # Aks the user the dimension of the board he wants to play
    @staticmethod
    def dim_board():
        return int(input("Attax\nChoose the dimension of the board: \n1-4*4 2-5*5 3-6*6")) 

    # Verifies if (x,y) is inside the board
    def inside(self, x, y):
        return (x>=0 and x<=self.N-1 and y>=0 and y<=self.N-1)

    # Verifies if mov is between two adjacents coordenates (at distances 1 or 2)
    @staticmethod
    def adjacent(mov, dist):
        return (abs(mov.xi-mov.xf)==dist and abs(mov.yi-mov.yf)<=dist or abs(mov.yi-mov.yf)==dist and abs(mov.xi-mov.xf)<=dist)
    
    # Indicates if a move is valid or not
    def is_valid_move(self, state, mov):
        if (not self.inside(mov.xi, mov.yi)) or not self.inside(mov.xf, mov.yf): # Verifies if the movement is inside the board
            return False # Out of the board
        if state[mov.yi][mov.xi]==mov.player and state[mov.yf][mov.xf]==0 and self.adjacent(mov, 1): # Verifies if the movement is valid
            mov.type = 0 # Expansion
            return True  # Expansion (move the piece)
        if state[mov.yi][mov.xi]==mov.player and state[mov.yf][mov.xf]==0 and self.adjacent(mov, 2): # Verifies if the movement is valid
            mov.type = 1 # Jump
            return True # Jump (move the piece)
        return False # Invalid movement

    # Multiplies the pieces
    def multiplies(self, state, mov):
        for dx in range(-1, 2): # Iterate over columns
            for dy in range(-1, 2): # Iterate over lines
                # Verifies if the movement is valid
                if mov.yf+dy >= 0 and mov.yf+dy < self.N and mov.xf+dx >= 0 and mov.xf+dx < self.N and state[mov.yf+dy][mov.xf+dx]==self.change_player(mov.player):
                    state[mov.yf+dy][mov.xf+dx]=mov.player  # Change the piece

    # Executes the movement 
    def get_next_state(self, state, mov): 
        state[mov.yf][mov.xf] = mov.player # Move the piece
        if mov.type==1: # Jump
            state[mov.yi][mov.xi] = 0 # Remove a piece
        self.multiplies(state, mov) # Multiply the pieces
        return state

    # Heuristic function that counts the number of pieces of a player
    def count_pieces(self, state, num):
        cp = 0 # Number of pieces
        i = 0 # Line
        while i<self.N: 
            j = 0 # Column
            while j<self.N: 
                if state[i][j]==num: # Verifies if the piece is from the player
                    cp += 1 # Increments the number of pieces
                j += 1 # Increments the column
            i += 1 # Increments the line
        return cp # Returns the number of pieces

    # Evaluation Function
    def evaluation(self, state, player):
        return self.count_pieces(state, player)-self.count_pieces(state, self.change_player(player)) # Returns the difference between the number of pieces of the player and the opponent

    # Determines every possible movement of a player
    def get_valid_moves(self, state, player):
        valid_moves = [] # List of valid movements

        for y in range(self.N):  # Iterate over lines (rows)
            for x in range(self.N):  # Iterate over columns
                if state[y][x] == player: # If the piece belongs to the player
                    # Check all adjacent cells for expansion moves
                    for dy in range(-1, 2): # Iterate over lines
                        for dx in range(-1, 2): # Iterate over columns
                            if dy == 0 and dx == 0: # Skip the current cell
                                continue 
                            new_x, new_y = x + dx, y + dy # New coordinates
                            # Check if the move is within the board and the destination is empty
                            if 0 <= new_x < self.N and 0 <= new_y < self.N and state[new_y][new_x] == 0:
                                valid_moves.append(f"{x}{y}_{new_x}{new_y}") # Add the movement to the list of valid movements

                    # Check cells within a 2-square range for jump moves
                    for dy in [-2, 2]: # Iterate over lines
                        for dx in [-2, -1, 0, 1, 2]: # Iterate over columns
                            new_x, new_y = x + dx, y + dy # New coordinates
                            if 0 <= new_x < self.N and 0 <= new_y < self.N and state[new_y][new_x] == 0:
                                valid_moves.append(f"{x}{y}_{new_x}{new_y}") # Add the movement to the list of valid movements
                    for dx in [-2, 2]: # Iterate over columns
                        for dy in [-1, 0, 1]: # Iterate over lines
                            new_x, new_y = x + dx, y + dy # New coordinates
                            if 0 <= new_x < self.N and 0 <= new_y < self.N and state[new_y][new_x] == 0:
                                valid_moves.append(f"{x}{y}_{new_x}{new_y}") # Add the movement to the list of valid movements

        return valid_moves # Returns the list of valid movements



    # Verify if the game is over 
    def get_value_and_terminated(self, state, player):
        valid_moves = self.get_valid_moves(state, player) # List of valid movements
        if len(valid_moves) > 0: # If there are valid movements
            return 0, None, False # The game is not over
        
        if self.evaluation(state, player)>0: # If the player has more pieces than the opponent
            return 1, player, True # The player wins
        
        if self.evaluation(state, player)<=0: # If the player has less pieces than the opponent
            return 1, self.change_player(player), True # The opponent wins
        
        return 0, None, True # The game is a draw

    # Change the perspective of the board
    def change_perspective(self, state, player): 
        return state * player  

    # Encode the state
    def get_encoded_state(self, state):
        encoded_state = np.stack(
            (state == -1, state == 0, state == 1) 
        ).astype(np.float32)

        return encoded_state 
